<a href="https://colab.research.google.com/github/Tech-savvyPavithra/Data_Science/blob/main/Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install dependencies
!pip install git+https://github.com/AliaksandrSiarohin/first-order-model
!pip install imageio imageio-ffmpeg

# Import necessary libraries
import imageio.v2 as imageio
import numpy as np
from skimage.transform import resize
import torch
import requests
import os
import yaml

# Define paths
source_image_path = "/content/drive/MyDrive/Colab Notebooks/My Workouts/test2.jpg"
driving_video_path = "/content/drive/MyDrive/Colab Notebooks/My Workouts/dance.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/My Workouts/save.mp4"
config_path = "/content/vox-256.yaml"
checkpoint_path = "/content/vox-cpk.pth.tar"

# Pull config file from GitHub
config_url = "https://raw.githubusercontent.com/AliaksandrSiarohin/first-order-model/main/config/vox-256.yaml"
if not os.path.exists(config_path):
    with open(config_path, 'wb') as f:
        f.write(requests.get(config_url).content)

# Pull checkpoint file from GitHub
checkpoint_url = "https://github.com/AliaksandrSiarohin/first-order-model/raw/main/vox-cpk.pth.tar"
if not os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'wb') as f:
        f.write(requests.get(checkpoint_url).content)

# Load checkpoints
def load_checkpoints(config_path, checkpoint_path):
    from demo import OcclusionAwareGenerator, KPDetector
    with open(config_path) as f:
        config = yaml.safe_load(f)
    generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
    kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    generator.load_state_dict(checkpoint['generator'])
    kp_detector.load_state_dict(checkpoint['kp_detector'])
    generator.to('cuda').eval()
    kp_detector.to('cuda').eval()
    return generator, kp_detector

# Load source image
source_image = imageio.imread(source_image_path)
source_image = resize(source_image, (256, 256))[..., :3]
source_image = torch.tensor(source_image[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2).to('cuda')

# Load driving video
reader = imageio.get_reader(driving_video_path)
driving_video = []
for frame in reader:
    frame = resize(frame, (256, 256))[..., :3]
    driving_video.append(frame)
driving_video = np.array(driving_video)
driving_video = torch.tensor(driving_video.astype(np.float32)).permute(0, 3, 1, 2).to('cuda')

# Perform image animation
generator, kp_detector = load_checkpoints(config_path, checkpoint_path)
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

# Save animation
imageio.mimsave(output_path, [img_as_ubyte(frame) for frame in predictions])

  Cloning https://github.com/AliaksandrSiarohin/first-order-model to /tmp/pip-req-build-jz4hy_s4
  Running command git clone --filter=blob:none --quiet https://github.com/AliaksandrSiarohin/first-order-model /tmp/pip-req-build-jz4hy_s4
  Resolved https://github.com/AliaksandrSiarohin/first-order-model to commit f4ff6da1ef5c0e6bcf6ec80324fab37c92193e84
ERROR: git+https://github.com/AliaksandrSiarohin/first-order-model does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


KeyError: 'model_params'